In [ ]:
'''
    units: 正整数，输出的维度；
    activation: 要采用的激活函数，一般保持默认 “tanh” 即可；
    recurrent_activation: 在每次循环的时候要采用的激活函数，默认为 “sigmoid”；
    dropout：用来抑制过拟合的一种方法，被应用于输入的阶段；
    recurrent_dropout：用来抑制过拟合的一种方法，被应用于循环的阶段；
    return_sequences：布尔值，是否返回一个完整的输出序列，还是只返回最后一个输出；
    return_state：布尔值，是否返回中间的隐含状态。
'''
tf.keras.layers.LSTM(
    units, activation='tanh', recurrent_activation='sigmoid', 
    dropout=0.0, recurrent_dropout=0.0, return_sequences=False,
    return_state=False
)

'''
对于 GRU ，我们可以使用以下的 API 实现。
可以看到 GRU 的 API 和 LSTM 的 API 的参数非常相似，而实际上他们的含义也相同。
    units: 正整数，GRU 输出的维度；
    activation: GRU 要采用的激活函数，一般保持默认 “tanh” 即可；
    recurrent_activation: GRU 在每次循环的时候要采用的激活函数，默认为 “sigmoid”；
    dropout：用来抑制过拟合的一种方法，被应用于 GRU 输入的阶段；
    recurrent_dropout：用来抑制过拟合的一种方法，被应用于 GRU 循环的阶段；
    return_sequences：布尔值，定义 GRU 是否返回一个完整的输出序列，还是只返回最后一个输出；
    return_state：布尔值，决定 GRU 是否返回中间的隐含状态。
'''
tf.keras.layers.GRU(
    units, activation='tanh', recurrent_activation='sigmoid', dropout=0.0,
    recurrent_dropout=0.0, return_sequences=False, return_state=False
)


In [ ]:
import tensorflow as tf
import numpy as np

# 定义基本参数
words_num = 10000
val_num = 12500
EPOCHS = 10
pad_max_length = 256
BATCH_SIZE = 64

# 获取数据:电影评价的数据集,文本数据集
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=words_num)
word_index = tf.keras.datasets.imdb.get_word_index()

# 添加特殊字符
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<pad>"] = 0
word_index["<start>"] = 1
word_index["<unknown>"] = 2
word_index["<unused>"] = 3

# 数据预处理
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, value=0, padding='post', maxlen=pad_max_length)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, value=0, padding='post', maxlen=pad_max_length)

# 划分训练集合与验证集合
x_val, x_train = train_data[:val_num], train_data[val_num:]
y_val, y_train = train_labels[:val_num], train_labels[val_num:]

# 模型构建
'''
该处的代码中，我们需要注意的一点是，
我们将 LSTM 用 tf.keras.layers.Bidirectional () 这一网络层包裹起来了。
顾名思义，这一个双向的网络层，通过将 LSTM 使用该网络层包裹，
LSTM 不仅仅能够学习到正向的信息，也能学习到反向的信息。
举个不恰当的例子，相当于人可以反着读书一样。
这可以增强循环神经网络提取信息的能力，提升模型的效果。

可以看到，我们的代码与之前的模型几乎完全相同，只是我们的网络模型采用了 LSTM（长短期记忆），
大家可以将这次运行的结果与之前运行的结果相比较。
'''
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(words_num, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练
history = model.fit(x_train, y_train, epochs=EPOCHS,
          batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

# 测试
results = model.evaluate(test_data, test_labels)

print(results)
